In [1]:
from pokemon_image import *
from affine_transform import *
from tqdm import tqdm

c:\Users\zhufe\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
data = load_image_data("images", "images/annotations.json")

In [3]:
def affine_transform_data(data):
    return [affine_transform_pokemon_image(i) for i in data]

In [4]:
TRANSFORMATIONS = 1

In [5]:
full_dataset = []
for _ in tqdm(range(TRANSFORMATIONS)):
    full_dataset.extend(affine_transform_data(data))
full_dataset += data

100%|██████████| 1/1 [00:00<00:00,  1.45it/s]


In [6]:
dataset = PokemonData(full_dataset)

In [4]:
import segmentation_models_pytorch as smp

model = smp.Unet('resnet34', encoder_weights='imagenet')

ModuleNotFoundError: No module named 'segmentation_models_pytorch'

In [17]:
import torch
import torch.nn as nn
from torchvision import models

# Load pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Freeze all the layers in the network
for param in model.parameters():
    param.requires_grad = False
    
model.avgpool = nn.Identity()
model.fc = nn.Identity()

# Output: Linear(in_features=2048, out_features=1000, bias=True)

In [1]:
from transformers import DetrForSegmentation

model = DetrForSegmentation.from_pretrained("facebook/detr-resnet-50-panoptic")

c:\Users\zhufe\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


AttributeError: module 'torch' has no attribute 'frombuffer'